In [ ]:
import os
import sys
import re
import tarfile
import time
import math
import numpy as np
import pandas as pd

In [5]:
out_path = "../corpus/novel_formated/"
corpus_name = "novel_segments2.tsv"

In [8]:
import csv
conv_data = []
with open(out_path+corpus_name, "r") as f:
    reader = csv.reader(f, delimiter="\t")
    conv_data = [row for row in reader]

In [11]:
conv_data[:4]

[['透子ちゃんは嬉しそうに、使ったガムテープの裏側を見せてくる『汚いって!モザイク必要なヤツだからそれ!』と突っ込む気力は今の俺には無くて。',
  'はい、神谷君、完成だよ!我ながらいい出来かも!',
  'こっちも終わったぞ、シュウイチ!見ろ!こんなにも毛が張り付いてるぞ!'],
 ['透子ちゃんは嬉しそうに、使ったガムテープの裏側を見せてくる『汚いって!モザイク必要なヤツだからそれ!』と突っ込む気力は今の俺には無くて。',
  '放心状態でそう呟き、ようやく椅子から解放された俺は、倒れそうになりながらその場から去ろうとしたんだ。',
  'こっちも終わったぞ、シュウイチ!見ろ!こんなにも毛が張り付いてるぞ!',
  'ああやっと終わったんだなこの悪夢が。'],
 ['透子ちゃんは嬉しそうに、使ったガムテープの裏側を見せてくる『汚いって!モザイク必要なヤツだからそれ!』と突っ込む気力は今の俺には無くて。',
  '放心状態でそう呟き、ようやく椅子から解放された俺は、倒れそうになりながらその場から去ろうとしたんだ。',
  'そこで花音ちゃんが両手広げて、俺の行く手を塞いできた。',
  'ああやっと終わったんだなこの悪夢が。',
  'ああ、待って神ちゃん、一回鏡見てよ!きっと目が覚めると思うからさ!'],
 ['透子ちゃんは嬉しそうに、使ったガムテープの裏側を見せてくる『汚いって!モザイク必要なヤツだからそれ!』と突っ込む気力は今の俺には無くて。',
  '放心状態でそう呟き、ようやく椅子から解放された俺は、倒れそうになりながらその場から去ろうとしたんだ。',
  'そこで花音ちゃんが両手広げて、俺の行く手を塞いできた。',
  'ああ、待って神ちゃん、一回鏡見てよ!きっと目が覚めると思うからさ!',
  'そんなこと言ってどうせアレでしょ?俺の顔がぐちゃぐちゃで『うわ、何だこの化け物!?』みたいな感じで目が覚めるヤツでしょ?']]

In [19]:
# 後半2つの発話のみを抽出して，どれくらいか

# conv_only_utt = []
# for conv in conv_data[:500]:
#     if len(conv) >=  5:
#         print(conv)

conv_only_utt = [conv[-3:] for conv in conv_data]

In [21]:
conv_only_utt[-40:]

[['叶がアキラと一緒に学校へ行くために下で待っている。', 'アキ姉ぇ、はやくー。', 'ほら、叶も待ってるぞ。'],
 ['最近作って全員からの評判が良かったミルフィーユカツを作る事にした。', 'ただいま楓太兄ぃ、今日はなに?', 'ミルフィーユカツだよ。'],
 ['最近作って全員からの評判が良かったミルフィーユカツを作る事にした。',
  'ミルフィーユカツだよ。',
  'ほんと!?アキ姉ぇ、今日はミルフィーユカツ!'],
 ['次第に帰ってくる姉妹達。', 'ほんと!?アキ姉ぇ、今日はミルフィーユカツ!', 'お、いいね。'],
 ['他愛もない雑談。',
  'そういえば、近所の猫が大量にいる家あるじゃないですか。あそこ、まーた増えてましたよ。',
  'まじ?夜中とか鳴き声がすごいんだよなぁ。'],
 ['愛花も真奈美さんも帰ってきて、全員で夕食に。', 'いただきまーす。', 'A!'],
 ['愛花も真奈美さんも帰ってきて、全員で夕食に。', 'A!', 'いやBだろこれ。'],
 ['愛花も真奈美さんも帰ってきて、全員で夕食に。', 'いやBだろこれ。', 'どう考えてもCだろう。'],
 ['クイズ番組を観ながら、それぞれが正解だと思うものを挙げていく。俺もAだと思う。', 'どう考えてもCだろう。', 'えー、お母さんもCだと思う。'],
 ['一箱で複数個はいってあるタイプのアイス。', 'アイスあるぞー。', 'あ、たべたーい。'],
 ['下の二人はそうもいかない。叶は日によるが、アキラは起こしに行かないと布団から出てこない。',
  '愛花、アキラと叶はまだ寝てる?',
  'そうだな。物音もしなかったから、おそらく。'],
 ['二人を起こしに部屋に向かう。', 'そうだな。物音もしなかったから、おそらく。', 'まったく夜ふかしばかりするからだ。'],
 ['むくりと叶が起き上がる。目をこすりながら「おはよぉ楓太兄ぃ」とあくび混じりに起床のご挨拶。',
  '叶、アキラ。朝だぞ、起きないと遅刻するぞ。',
  'んはぁい。'],
 ['むくりと叶が起き上がる。目をこすりながら「おはよぉ楓太兄ぃ」とあくび混じりに起床のご挨拶。',
  'んはぁい。',
  'もう朝ごはん用意してるから、下降りて食べな。'],
 ['叶は案外目が